# 自动处理越南语没有成功翻译的字段

In [3]:
pip install openai python-dotenv

  Using cached openai-1.58.1-py3-none-any.whl.metadata (27 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached anyio-4.7.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.8.2-cp310-cp310-win_amd64.whl.metadata (5.3 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached openai-1.58.1-py3-none-any.whl (454 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Using cached anyio-4.7.0-py3-none-any.whl (93 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
Using cached jiter-0.8.2-cp310-cp310-win_amd64.whl (204 kB)
Using cached sniffio-1.3.1-py3-none-any.whl 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import re

from openai import OpenAI
from dotenv import load_dotenv

def contains_chinese(text):
    # 使用正则表达式匹配中文字符
    return bool(re.search(r'[\u4e00-\u9fff]', text))

load_dotenv()
client = OpenAI()
original_language = "Chinese"
target_language = "Korean"
def generate_text(index, data):
    if data["module"] == "Error: Connection error.":
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": f"You are an expert in translating {original_language} to {target_language} for ERP systems. Your task is to translate markdown-formatted text from {original_language} to {target_language}. Preserving its formatting without adding extra content."},
                {"role": "user", "content": str(data['value'])}
            ],
            temperature=0
        )
        return index, completion.choices[0].message.content
    else:
        return index, str(data["module"])
    
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd

filenames = [
    r"C:\Users\wangz\Downloads\翻译韩语从B到C列-0116.xlsx",
]
for filename in filenames:
    to_fix = pd.read_excel(filename)

    with ThreadPoolExecutor(max_workers=1000) as executor:
        futures = {executor.submit(generate_text, index, row) for index, row in to_fix.iterrows()}

        for future in tqdm(as_completed(futures), total=len(futures)):
            index, output = future.result()

            if output is not None:
                to_fix.at[index, "module"] = output
    # 保存最终结果到 Excel 文件
    to_fix.to_excel(filename.replace(".xlsx", "_translated.xlsx"), index=False)

100%|██████████| 190793/190793 [00:05<00:00, 33760.82it/s]


In [10]:
to_fix.to_csv(r"C:\Users\wangz\Downloads\未翻译词条_1.csv", index=False)

In [6]:
to_fix = pd.read_excel(r"C:\Users\wangz\Downloads\未翻译词条.xlsx")

# 第二次检测修复字符串

In [7]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()
original_language = "English"
target_language = "Vietnamese"
def generate_text(index, data):
    try:
        if str(data['备注']).strip() == 'SPECIAL VALUE' or str(data['备注']).strip() == 'SPECIAL VALUE, INCLUDE UPPERCASE LETTER' or str(data['备注']).strip() == '':
            return index, str(data['待翻译(译)']).strip()
        if str(data['参考语言(英文)']).strip().lower() != str(data['简体中文(源)']).strip().lower() and str(data['参考语言(英文)']).strip().lower() == str(data['待翻译(译)']).strip().lower():
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": f"You are an expert in translating {original_language} to {target_language} for ERP systems. Your task is to translate markdown-formatted text from {original_language} to {target_language}. Preserving its formatting without adding extra content."},
                    {"role": "user", "content": str(data['参考语言(英文)'])}
                ],
                temperature=0
            )
            return index, completion.choices[0].message.content
        if str(data['参考语言(英文)']).strip().lower() == str(data['简体中文(源)']).strip().lower():
            return index, str(data['参考语言(英文)']).strip()
        else:
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "user", "content": "Translate English: The received L/G %s is not submitted \nTo Spanish: El L/G recibido no se ha enviado. \nPlease fix the translate error, because %s is missing, translate it directly and without adding any extra content."},
                    {"role": "assistant", "content": "El L/G recibido %s no se ha enviado."},
                    {"role": "user", "content": f"Translate {original_language}: {str(data['参考语言(英文)'])} \nTo {target_language}: {str(data['待翻译(译)'])} \n Please fix the translate error, Because {str(data['备注'])}. translate it directly and without adding any extra content."}
                ],
                temperature=0
            )
            return index, completion.choices[0].message.content
    except:
        return index, str(data['待翻译(译)']).strip()

In [8]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
to_fix = pd.read_excel(r"D:\Projects\ai-translator\special value\first_check\processed_files\multilangInitData YS全量词条 20241119 Special Value_中_越.xlsx")

with ThreadPoolExecutor(max_workers=1000) as executor:
    futures = {executor.submit(generate_text, index, row) for index, row in to_fix.iterrows()}

    for future in tqdm(as_completed(futures), total=len(futures)):
        index, output = future.result()

        if output is not None:
            to_fix.at[index, f'待翻译(译)'] = output

to_fix['备注'] = ""
# 保存最终结果到 Excel 文件
to_fix.to_excel(r"D:\Projects\ai-translator\special value\first_check\processed_files\multilangInitData YS全量词条 20241119 Special Value_中_越_processed.xlsx", index=False)

100%|██████████| 124202/124202 [02:26<00:00, 849.76it/s] 


# 文件合并

In [1]:
import pandas as pd

# 定义要合并的 Excel 文件列表
excel_files = [
    r"C:\Users\wangz\Downloads\processed\part_1_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_2_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_3_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_4_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_5_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_6_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_7_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_8_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_9_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_10_translated.xlsx",
    r"C:\Users\wangz\Downloads\processed\part_11_translated.xlsx",
]

# 读取并合并所有 Excel 文件
df_list = [pd.read_excel(file) for file in excel_files]
merged_df = pd.concat(df_list, ignore_index=True)

# 保存合并后的 DataFrame 到新的 Excel 文件
merged_df.to_excel(r"D:\Projects\ai-translator\src\multilangInitData20250210空_translated.xlsx", index=False)


FileCreateError: [WinError 2] 系统找不到指定的文件。: 'C:\\Users\\wangz\\AppData\\Local\\Temp\\tmp2dbjpq31'

# 文件拆分

In [1]:
import os
import pandas as pd

def split_excel(input_file, output_folder, chunk_size=100000):
    """
    将一个Excel文件按指定行数拆分为多个Excel文件，并存储到指定文件夹。

    Args:
        input_file (str): 输入的Excel文件路径。
        output_folder (str): 输出文件夹路径。
        chunk_size (int): 每个拆分Excel文件包含的最大行数，默认10万。
    """
    # 检查输出文件夹是否存在，不存在则创建
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # 读取Excel文件（假设默认是Sheet1）
    print("正在读取Excel文件...")
    excel_data = pd.read_excel(input_file, sheet_name=0)

    # 获取总行数
    total_rows = excel_data.shape[0]
    print(f"Excel文件总行数: {total_rows} 行")

    # 按chunk_size进行分割
    for i in range(0, total_rows, chunk_size):
        chunk = excel_data.iloc[i:i + chunk_size]  # 提取数据块
        output_file = os.path.join(output_folder, f"part_{i // chunk_size + 1}.xlsx")
        chunk.to_excel(output_file, index=False, engine='openpyxl')  # 保存为Excel
        print(f"已保存: {output_file}")
    
    print("拆分完成！")

# 示例用法
if __name__ == "__main__":
    input_file = r"C:\Users\wangz\Downloads\multilangInitData_副本.xlsx"  # 输入文件路径
    output_folder = r"D:\Projects\ai-translator\src\multilangInitData_副本"  # 输出文件夹路径
    split_excel(input_file, output_folder)


正在读取Excel文件...


d:\Programs\Anaconda3\envs\ai-trans\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Excel文件总行数: 1048128 行
已保存: D:\Projects\ai-translator\src\multilangInitData_副本\part_1.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData_副本\part_2.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData_副本\part_3.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData_副本\part_4.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData_副本\part_5.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData_副本\part_6.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData_副本\part_7.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData_副本\part_8.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData_副本\part_9.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData_副本\part_10.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData_副本\part_11.xlsx
拆分完成！


# 友互通翻译

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import zhconv

load_dotenv()
client = OpenAI()
target_columns = [
    "印尼语",
    "匈牙利语",
    "葡萄牙语",
    "泰语",
    "土耳其语",
    "越南语",
    "俄语",
    "阿拉伯语",
    "芬兰语",
    "丹麦语",
    "荷兰语",
    "波兰语",
    "法语",
    "德语",
    "日语",
    "挪威语",
    "希伯来语",
    "韩语",
    "西班牙语",
    "捷克语",
    "意大利语",
    "瑞典语",
    "希腊语",
    "马来语",
    "斯洛伐克语",
    "柬埔寨语",
    "罗马尼亚语",
    "克罗地亚语",
    "乌兹别克语",
    "缅甸语"
]

simple_column_name = "简体中文(译)"
english_column_name = "English"
trans_column_name = "繁體中文"

for target_column in target_columns:
    print(target_column)
    dictionary = {}

    def generate_text(index, data):
        if not pd.isnull(data[target_column]):
            dictionary[str(data[english_column_name])] = data[target_column]
            return index, data[target_column]
        if str(data[english_column_name]) in dictionary.keys():
            return index, dictionary[str(data[english_column_name])]
        if target_column == trans_column_name:
            return index, zhconv.convert(str(data[simple_column_name]), 'zh-tw')
        completion = client.chat.completions.create(
            model='gpt-4o',
            messages=[
                {"role": "user", "content": f"Translate the following sentence or word from English to {simple_column_name}: {str(data[english_column_name])}, please directly translate it and do not output any extra content"},
                {"role": "assistant", "content": str(data[simple_column_name])},
                {"role": "user", "content": f"Translate the following sentence or word from English to {trans_column_name}: {str(data[english_column_name])}, please directly translate it and do not output any extra content"},
                {"role": "assistant", "content": str(data[trans_column_name])},
                {"role": "user", "content": f"Translate the following sentence or word from English to {target_column}: {str(data[english_column_name])}, please directly translate it and do not output any extra content"}
            ],
            # messages=[
            #     {"role": "user", "content": f"Translate the following sentence from Chinese to {english_column_name}: {str(data[simple_column_name])}, please directly translate it and do not output any extra content"},
            #     {"role": "assistant", "content": str(data[english_column_name])},
            #     {"role": "user", "content": f"Translate the following sentence from Chinese to {trans_column_name}: {str(data[simple_column_name])}, please directly translate it and do not output any extra content"},
            #     {"role": "assistant", "content": str(data[trans_column_name])},
            #     {"role": "user", "content": f"Translate the following sentence from Chinese to {target_column}: {str(data[simple_column_name])}, please directly translate it and do not output any extra content"}
            # ],
            temperature=0
        )
        dictionary[str(data[english_column_name])] = completion.choices[0].message.content
        return index, completion.choices[0].message.content

    from tqdm import tqdm
    from concurrent.futures import ThreadPoolExecutor, as_completed
    import pandas as pd
    to_fix = pd.read_excel(r"C:\Users\wangz\Downloads\友户通-新增词条-需要翻译-33种-20250411-MT.xlsx")

    with ThreadPoolExecutor(max_workers=1000) as executor:
        futures = {executor.submit(generate_text, index, row) for index, row in to_fix.iterrows()}

        for future in tqdm(as_completed(futures), total=len(futures)):
            index, output = future.result()

            if output is not None:
                to_fix.at[index, target_column] = output

    # 保存最终结果到 Excel 文件
    to_fix.to_excel(r"C:\Users\wangz\Downloads\友户通-新增词条-需要翻译-33种-20250411-MT.xlsx", index=False)

印尼语


100%|██████████| 6/6 [00:01<00:00,  4.11it/s]


匈牙利语


100%|██████████| 6/6 [00:00<00:00, 10.14it/s]


葡萄牙语


...

100%|██████████| 6/6 [00:00<00:00,  8.53it/s]


乌兹别克语


100%|██████████| 6/6 [00:00<00:00,  7.62it/s]


缅甸语


100%|██████████| 6/6 [00:01<00:00,  5.44it/s]


# PDF翻译

In [4]:
from dotenv import load_dotenv
load_dotenv()

!pdf2zh "C:\Users\wangz\Downloads\SHOW-O：ONE SINGLE TRANSFORMER TO UNIFY.pdf" -li en -lo zh -s openai

not in git repo
[04/01/25 10:50:09] INFO     INFO:babeldoc.assets.assets:doclayou assets.py:196
                             t onnx model not found or corrupted,              
                             downloading...                                    
[04/01/25 10:50:17] INFO     INFO:babeldoc.assets.assets:Download assets.py:207
                             doclayout onnx model from                         
                             huggingface success                               
Namespace(files=['C:\\Users\\wangz\\Downloads\\SHOW-O：ONE SINGLE TRANSFORMER TO UNIFY.pdf'], debug=False, pages=None, vfont='', vchar='', lang_in='en', lang_out='zh', service='openai', output='', thread=4, interactive=False, share=False, flask=False, celery=False, authorized=None, prompt=None, compatible=False, onnx=None, serverport=None, dir=False, config=None, babeldoc=False, skip_subset_fonts=False, ignore_cache=False)
[04/01/25 10:50:18] INFO     INFO:babeldoc.assets.assets:Font     assets.py:26


100%|██████████| 27/27 [04:01<00:00,  8.96s/it]


In [2]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/", api_key="")
target_column = "印尼语"

def generate_text(index, data):
    if not pd.isnull(data[target_column]):
        return index, data[target_column]
    completion = client.chat.completions.create(
        model='gemini-1.5-flash',
        messages=[
            {"role": "user", "content": f"Translate English to Chinese: {str(data['English(译)'])}, please directly translate it and do not output any extra content"},
            {"role": "assistant", "content": str(data['简体中文(源)'])},
            {"role": "user", "content": f"Translate English to Traditional Chinese: {str(data['English(译)'])}, please directly translate it and do not output any extra content"},
            {"role": "assistant", "content": str(data['繁体中文（译）'])},
            {"role": "user", "content": f"Translate English to {target_column}: {str(data['English(译)'])}, please directly translate it and do not output any extra content"}
        ],
        temperature=0
    )
    return index, completion.choices[0].message.content

from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
to_fix = pd.read_excel(r"D:\Projects\ai-translator\src\友互通\友互通-全.xlsx")

with ThreadPoolExecutor(max_workers=1000) as executor:
    futures = {executor.submit(generate_text, index, row) for index, row in to_fix.iterrows()}

    for future in tqdm(as_completed(futures), total=len(futures)):
        index, output = future.result()

        if output is not None:
            to_fix.at[index, target_column] = output

# 保存最终结果到 Excel 文件
to_fix.to_excel(r"D:\Projects\ai-translator\src\友互通\友互通-全.xlsx", index=False)

(0, 'matahari\n')

# 翻译123种语言

初翻

In [2]:
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import threading

# Load environment variables and set OpenAI API key
load_dotenv()
client = OpenAI()
# Read the Excel file into a pandas DataFrame
data = pd.read_excel(r"C:\Users\wangz\Downloads\20241203 123种语种翻译数据.xlsx")

# Define the columns and rows to process
columns = data.columns.to_list()[4:]  # Exclude the first 4 columns
rows = data.index[2:]  # Start from the third row

# Prepare the list of tasks for translation
tasks = []
for column in columns:
    for row_index in rows:
        text_to_translate = data.at[row_index, 'zh_CN']
        tasks.append((row_index, column, text_to_translate))

# Initialize a lock for thread-safe DataFrame operations
data_lock = threading.Lock()
max_workers = 2000  # Adjust the number of threads as needed

# Define the function to translate a single cell
def translate_cell(task):
    row_index, column, text_to_translate = task
    messages = [
        {"role": "user", "content": f"Translate '简体中文' to {column}, translate it directly and without adding any extra content."},
        {"role": "assistant", "content": str(data[column][1])},
        {"role": "user", "content": f"Translate '{text_to_translate}' to {column}, translate it directly and without adding any extra content."}
    ]
    try:
        completion =  client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0
        )
        translated_text = completion.choices[0].message.content
        # Write the translated text back to the DataFrame in a thread-safe manner
        with data_lock:
            data.at[row_index, column] = translated_text
        return (row_index, column, translated_text)
    except Exception as e:
        print(f"Error translating row {row_index}, column {column}: {e}")
        return (row_index, column, None)

# Execute the translation tasks using a ThreadPoolExecutor and display a progress bar
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(translate_cell, task) for task in tasks]
    for future in tqdm(as_completed(futures), total=len(futures)):
        pass  # The results are handled within the translate_cell function

# Save the translated DataFrame to a new Excel file
data.to_excel("translated_data.xlsx", index=False)

100%|██████████| 14640/14640 [02:03<00:00, 118.87it/s]


In [1]:
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import threading

# Load environment variables and set OpenAI API key
load_dotenv()
client = OpenAI()
# Read the Excel file into a pandas DataFrame
data = pd.read_excel(r"C:\Users\wangz\Documents\WeChat Files\wxid_eet1drkwse4k22\FileStorage\File\2024-12\20241203 123种语种翻译数据-GPT 确认.xlsx")

# Define the columns and rows to process
columns = data.columns.to_list()[4:]  # Exclude the first 4 columns
rows = data.index[2:]  # Start from the third row

# Prepare the list of tasks for translation
tasks = []
for column in columns:
    for row_index in rows:
        text_to_translate = data.at[row_index, 'zh_CN']
        if text_to_translate == '挪威语':
            tasks.append((row_index, column, text_to_translate))
            break  # Only translate the first occurrence

# Initialize a lock for thread-safe DataFrame operations
data_lock = threading.Lock()
max_workers = 2000  # Adjust the number of threads as needed

# Define the function to translate a single cell
def translate_cell(task):
    row_index, column, text_to_translate = task
    messages = [
        {"role": "user", "content": f"Translate '简体中文' to {column}, translate it directly and without adding any extra content."},
        {"role": "assistant", "content": str(data[column][1])},
        {"role": "user", "content": f"Translate '{text_to_translate}' to {column}, translate it directly and without adding any extra content."}
    ]
    try:
        completion =  client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            temperature=0
        )
        translated_text = completion.choices[0].message.content
        # Write the translated text back to the DataFrame in a thread-safe manner
        with data_lock:
            data.at[row_index, column] = translated_text
        return (row_index, column, translated_text)
    except Exception as e:
        print(f"Error translating row {row_index}, column {column}: {e}")
        return (row_index, column, None)

# Execute the translation tasks using a ThreadPoolExecutor and display a progress bar
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(translate_cell, task) for task in tasks]
    for future in tqdm(as_completed(futures), total=len(futures)):
        pass  # The results are handled within the translate_cell function

# Save the translated DataFrame to a new Excel file
data.to_excel("translated_data.xlsx", index=False)

100%|██████████| 120/120 [00:03<00:00, 32.88it/s] 


# 精翻

In [6]:
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import threading

# Load environment variables and set OpenAI API key
load_dotenv()
client = OpenAI()
# Read the Excel file into a pandas DataFrame
data = pd.read_excel(r"D:\Projects\ai-translator\src\translated_data.xlsx")

# Define the columns and rows to process
columns = data.columns.to_list()[4:]  # Exclude the first 4 columns
rows = data.index[2:]  # Start from the third row

# Prepare the list of tasks for translation
tasks = []
for column in columns:
    for row_index in rows:
        text_to_translate = data.at[row_index, 'zh_CN']
        tasks.append((row_index, column, text_to_translate))

# Initialize a lock for thread-safe DataFrame operations
data_lock = threading.Lock()
max_workers = 2000  # Adjust the number of threads as needed

# Define the function to translate a single cell
def translate_cell(task):
    row_index, column, text_to_translate = task
    if text_to_translate.strip() == data.at[row_index, column].strip():
        messages = [
            {"role": "user", "content": f"Translate '简体中文' to {column}, translate it directly and without adding any extra content."},
            {"role": "assistant", "content": str(data[column][1])},
            {"role": "user", "content": f"Translate '{text_to_translate}' to {column}, translate it directly and without adding any extra content."}
        ]
        try:
            completion =  client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0
            )
            translated_text = completion.choices[0].message.content
            # Write the translated text back to the DataFrame in a thread-safe manner
            with data_lock:
                data.at[row_index, column] = translated_text
            return (row_index, column, translated_text)
        except Exception as e:
            print(f"Error translating row {row_index}, column {column}: {e}")
            return (row_index, column, None)
    else:
        return (row_index, column, data.at[row_index, column])

# Execute the translation tasks using a ThreadPoolExecutor and display a progress bar
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(translate_cell, task) for task in tasks]
    for future in tqdm(as_completed(futures), total=len(futures)):
        pass  # The results are handled within the translate_cell function

# Save the translated DataFrame to a new Excel file
data.to_excel("translated_data_1.xlsx", index=False)

100%|██████████| 14640/14640 [00:01<00:00, 14606.36it/s] 


# 统计文件夹下的文件数

In [14]:
import os

def count_files(folder_path):
    total_files = 0
    for root, dirs, files in os.walk(folder_path):
        total_files += len(files)  # 统计当前目录的文件数量
    return total_files

# 输入文件夹路径
folder_path = r"C:\Users\wangz\Documents\WeChat Files\wxid_eet1drkwse4k22\FileStorage\File\2024-12\高级版2312帮助文档（English）\高级版2312帮助文档（English）"
file_count = count_files(folder_path)
print(f"文件总数为: {file_count}")


文件总数为: 5001


# 翻译越南语

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import zhconv

load_dotenv()
client = OpenAI()
target_columns = [
    "目标语言"
]

for target_column in target_columns:
    dictionary = {}

    def generate_text(index, data):
        if not pd.isnull(data[target_column]):
            dictionary[str(data['English(译)'])] = data[target_column]
            return index, data[target_column]
        if str(data['English(译)']) in dictionary.keys():
            return index, dictionary[str(data['English(译)'])]
        if target_column == "繁体中文（译）":
            return index, zhconv.convert(str(data['简体中文(源)']), 'zh-tw')
        completion = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=[
                {"role": "user", "content": f"Translate English to Chinese: {str(data['English(译)'])}, please directly translate it and do not output any extra content"},
                {"role": "assistant", "content": str(data['简体中文(源)'])},
                {"role": "user", "content": f"Translate English to Traditional Chinese: {str(data['English(译)'])}, please directly translate it and do not output any extra content"},
                {"role": "assistant", "content": str(data["繁体中文（译）"])},
                {"role": "user", "content": f"Translate English to {target_column}: {str(data['English(译)'])}, please directly translate it and do not output any extra content"}
            ],
            temperature=0
        )
        dictionary[str(data['English(译)'])] = completion.choices[0].message.content
        return index, completion.choices[0].message.content

    from tqdm import tqdm
    from concurrent.futures import ThreadPoolExecutor, as_completed
    import pandas as pd
    to_fix = pd.read_excel(r"C:\Users\wangz\Downloads\前端词条补充翻译-20241227\TINPERNEXTPRO-FE_en_US20241226203251.xlsx")

    with ThreadPoolExecutor(max_workers=1000) as executor:
        futures = {executor.submit(generate_text, index, row) for index, row in to_fix.iterrows()}

        for future in tqdm(as_completed(futures), total=len(futures)):
            index, output = future.result()

            if output is not None:
                to_fix.at[index, target_column] = output

    # 保存最终结果到 Excel 文件
    to_fix.to_excel(r"C:\Users\wangz\Downloads\前端词条补充翻译-20241227\TINPERNEXTPRO-FE_en_US20241226203251.xlsx", index=False)

# 去重

In [1]:
import pandas as pd

# 读取 Excel 文件
file_path = r"C:\Users\wangz\Desktop\新建 XLSX 工作表.xlsx"  # 替换为你的文件路径
df = pd.read_excel(file_path)

# 删除 Chinese 列中的重复项，仅保留最后一个
df_cleaned = df.drop_duplicates(subset="Chinese", keep="last")

# 保存清理后的数据到新 Excel 文件
output_path = "Cleaned_Chinese_Column.xlsx"
df_cleaned.to_excel(output_path, index=False)

print(f"清理后的文件已保存为: {output_path}")


清理后的文件已保存为: Cleaned_Chinese_Column.xlsx


In [3]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{
        "role": "user",
        "content": "Who are you?"
    }],
    temperature=0,
    logprobs=True,
    top_p=1
)

completion.choices[0].message.content

'I am an AI language model created by OpenAI, designed to assist with a wide range of questions and tasks by providing information, answering queries, and engaging in conversation. How can I help you today?'

In [15]:
import numpy as np

logprobs = [token.logprob for token in completion.choices[0].logprobs.content]
probs = np.exp(logprobs)
geo_mean_confidence = np.prod(probs) ** (1 / len(probs))

In [17]:
float(geo_mean_confidence)

0.9011096809216835

# PDF2DOCX

In [6]:
import os
from pdf2docx import Converter

def convert_pdf_folder_to_docx(input_folder, output_folder):
    # 创建输出文件夹（如果不存在）
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 遍历输入文件夹中的所有 PDF 文件
    for filename in os.listdir(input_folder):
        if filename.lower().endswith('.pdf'):
            pdf_file = os.path.join(input_folder, filename)
            docx_file = os.path.join(output_folder, filename.replace('.pdf', '.docx'))
            
            # 转换 PDF 为 DOCX
            print(f"Converting {filename}...")
            try:
                cv = Converter(pdf_file)
                cv.convert(docx_file)
                cv.close()
                print(f"Successfully converted: {filename} -> {docx_file}")
            except Exception as e:
                print(f"Failed to convert {filename}: {e}")

    print("All files have been processed.")

# 输入 PDF 文件夹路径和输出 DOCX 文件夹路径
input_folder = r"C:\Users\wangz\Downloads\万华项目文档-刘坤林\02-24客开赋能"  # 替换为你的 PDF 文件夹路径
output_folder = r"C:\Users\wangz\Downloads\万华项目文档-刘坤林\02-24客开赋能"  # 替换为输出 DOCX 的文件夹路径

# 执行转换
convert_pdf_folder_to_docx(input_folder, output_folder)


[INFO] Start to convert C:\Users\wangz\Downloads\万华项目文档-刘坤林\02-24客开赋能\03-YonBIP旗舰版专业开发-开发介绍-数字化工作台介绍（含开发者中心和YMS介绍）.pdf
[INFO] [1/4] Opening document...


[INFO] [2/4] Analyzing document...


Converting 03-YonBIP旗舰版专业开发-开发介绍-数字化工作台介绍（含开发者中心和YMS介绍）.pdf...


[INFO] [3/4] Parsing pages...
[INFO] (1/45) Page 1
[INFO] (2/45) Page 2
[INFO] (3/45) Page 3
[INFO] (4/45) Page 4
[INFO] (5/45) Page 5
[INFO] (6/45) Page 6
[INFO] (7/45) Page 7
[INFO] (8/45) Page 8
[INFO] (9/45) Page 9
[INFO] (10/45) Page 10
[INFO] (11/45) Page 11
[INFO] (12/45) Page 12
[INFO] (13/45) Page 13
[INFO] (14/45) Page 14
[INFO] (15/45) Page 15
[INFO] (16/45) Page 16
[INFO] (17/45) Page 17
[INFO] (18/45) Page 18
[INFO] (19/45) Page 19
[INFO] (20/45) Page 20
[INFO] (21/45) Page 21
[INFO] (22/45) Page 22
[INFO] (23/45) Page 23
[INFO] (24/45) Page 24
[INFO] (25/45) Page 25
[INFO] (26/45) Page 26
[INFO] (27/45) Page 27
[INFO] (28/45) Page 28
[INFO] (29/45) Page 29
[INFO] (30/45) Page 30
[INFO] (31/45) Page 31
[INFO] (32/45) Page 32
[INFO] (33/45) Page 33
[INFO] (34/45) Page 34
[INFO] (35/45) Page 35
[INFO] (36/45) Page 36
[INFO] (37/45) Page 37
[INFO] (38/45) Page 38
[INFO] (39/45) Page 39
[INFO] (40/45) Page 40
[INFO] (41/45) Page 41
[INFO] (42/45) Page 42
[INFO] (43/45) Page 43

Successfully converted: 03-YonBIP旗舰版专业开发-开发介绍-数字化工作台介绍（含开发者中心和YMS介绍）.pdf -> C:\Users\wangz\Downloads\万华项目文档-刘坤林\02-24客开赋能\03-YonBIP旗舰版专业开发-开发介绍-数字化工作台介绍（含开发者中心和YMS介绍）.docx
Converting 07-YonBIP旗舰版专业开发-开发介绍-客开介绍.pdf...


...

Successfully converted: 18-YonBIP旗舰版专业开发-开发指南-扩展开发-扩展开发介绍.pdf -> C:\Users\wangz\Downloads\万华项目文档-刘坤林\02-24客开赋能\18-YonBIP旗舰版专业开发-开发指南-扩展开发-扩展开发介绍.docx
Converting 23-YonBIP旗舰版专业开发-开发指南-移动端开发-组件扩展开发.pdf...


[INFO] [3/4] Parsing pages...
[INFO] (1/28) Page 1
[INFO] (2/28) Page 2
[INFO] (3/28) Page 3
[INFO] (4/28) Page 4
[INFO] (5/28) Page 5
[INFO] (6/28) Page 6
[INFO] (7/28) Page 7
[INFO] (8/28) Page 8
[INFO] (9/28) Page 9
[INFO] (10/28) Page 10
[INFO] (11/28) Page 11
[INFO] (12/28) Page 12
[INFO] (13/28) Page 13
[INFO] (14/28) Page 14
[INFO] (15/28) Page 15
[INFO] (16/28) Page 16
[INFO] (17/28) Page 17
[INFO] (18/28) Page 18
[INFO] (19/28) Page 19
[INFO] (20/28) Page 20
[INFO] (21/28) Page 21
[INFO] (22/28) Page 22
[INFO] (23/28) Page 23
[INFO] (24/28) Page 24
[INFO] (25/28) Page 25
[INFO] (26/28) Page 26
[INFO] (27/28) Page 27
[INFO] (28/28) Page 28
[INFO] [4/4] Creating pages...
[INFO] (1/28) Page 1
[INFO] (2/28) Page 2
[INFO] (3/28) Page 3
[INFO] (4/28) Page 4
[INFO] (5/28) Page 5
[INFO] (6/28) Page 6
[INFO] (7/28) Page 7
[INFO] (8/28) Page 8
[INFO] (9/28) Page 9
[INFO] (10/28) Page 10
[INFO] (11/28) Page 11
[INFO] (12/28) Page 12
[INFO] (13/28) Page 13
[INFO] (14/28) Page 14
[INFO] (1

Successfully converted: 23-YonBIP旗舰版专业开发-开发指南-移动端开发-组件扩展开发.pdf -> C:\Users\wangz\Downloads\万华项目文档-刘坤林\02-24客开赋能\23-YonBIP旗舰版专业开发-开发指南-移动端开发-组件扩展开发.docx
Converting 24-YonBIP旗舰版专业开发-开发指南-移动端开发-打包.pdf...


[INFO] [3/4] Parsing pages...
[INFO] (1/30) Page 1
[INFO] (2/30) Page 2
[INFO] (3/30) Page 3
[INFO] (4/30) Page 4
[INFO] (5/30) Page 5
[INFO] (6/30) Page 6
[INFO] (7/30) Page 7
[INFO] (8/30) Page 8
[INFO] (9/30) Page 9
[INFO] (10/30) Page 10
[INFO] (11/30) Page 11
[INFO] (12/30) Page 12
[INFO] (13/30) Page 13
[INFO] (14/30) Page 14
[INFO] (15/30) Page 15
[INFO] (16/30) Page 16
[INFO] (17/30) Page 17
[INFO] (18/30) Page 18
[INFO] (19/30) Page 19
[INFO] (20/30) Page 20
[INFO] (21/30) Page 21
[INFO] (22/30) Page 22
[INFO] (23/30) Page 23
[INFO] (24/30) Page 24
[INFO] (25/30) Page 25
[INFO] (26/30) Page 26
[INFO] (27/30) Page 27
[INFO] (28/30) Page 28
[INFO] (29/30) Page 29
[INFO] (30/30) Page 30
[INFO] [4/4] Creating pages...
[INFO] (1/30) Page 1
[INFO] (2/30) Page 2
[INFO] (3/30) Page 3
[INFO] (4/30) Page 4
[INFO] (5/30) Page 5
[INFO] (6/30) Page 6
[INFO] (7/30) Page 7
[INFO] (8/30) Page 8
[INFO] (9/30) Page 9
[INFO] (10/30) Page 10
[INFO] (11/30) Page 11
[INFO] (12/30) Page 12
[INFO] (1

Successfully converted: 24-YonBIP旗舰版专业开发-开发指南-移动端开发-打包.pdf -> C:\Users\wangz\Downloads\万华项目文档-刘坤林\02-24客开赋能\24-YonBIP旗舰版专业开发-开发指南-移动端开发-打包.docx
All files have been processed.


# 术语表合并

In [3]:
import pandas as pd

original_path = r"D:\Projects\ai-translator\src\models\API\chatgpt-4o-mini\glossary.xlsx"
new_path = r"C:\Users\wangz\Downloads\用友技术类常用术语表 for MT-20250320.xlsx"
output_path = "glossary.xlsx"

def merge_translation_tables(original_path, new_path, output_path, original_language, target_language):
    # 读取原始术语表和新增术语表
    original_df = pd.read_excel(original_path)
    new_df = pd.read_excel(new_path)
    
    # 确保列名存在
    if original_language not in original_df.columns or target_language not in original_df.columns:
        raise ValueError("Original language and target language columns must exist in the original table")
    if original_language not in new_df.columns or target_language not in new_df.columns:
        raise ValueError("Original language and target language columns must exist in the new table")
    
    # 过滤掉原始术语为空的行
    original_df = original_df.dropna(subset=[original_language])
    new_df = new_df.dropna(subset=[original_language])
    
    # 找出新增的术语（原术语表中不存在的）
    existing_terms = set(original_df[original_language].astype(str))
    filtered_new_df = new_df[~new_df[original_language].astype(str).isin(existing_terms)]
    
    # 合并数据表
    merged_df = pd.concat([original_df, filtered_new_df], ignore_index=True)
    
    # 保存合并后的数据
    merged_df.to_excel(output_path, index=False)
    
    print(f"术语表已合并，保存至: {output_path}")
    return merged_df

merge_translation_tables(original_path, new_path, output_path, "Chinese", "English")

术语表已合并，保存至: new_glossary.xlsx


,Chinese,English,Thai
0,会计服务,Accounting,บริการด้านบัญชี
1,企业绩效,Enterprise Performance Management,ผลการดำเนินธุรกิจ
2,业财综合分析,Financial Analytics,การวิเคราะห์การเงินแบบบูรณาการ
3,数智合同,Contract Lifecycle Management,สัญญาอัจฉริยะดิจิทัล
4,采购供应,Material Mgmt,การจัดหาซัพพลาย
...,...,...,...
5048,缓存机制,Caching Mechanism,NaN
5049,管理类角色,Administrative Role,NaN
5050,业务类角色,Business Role,NaN
5051,应用发布,Application Release,NaN


# 阿语处理，填补空缺

In [3]:
import os
from openpyxl import load_workbook

# 文件列表
file_paths = [
    r"C:\Users\wangz\Downloads\multilangInitData_副本\part_1.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData_副本\part_2.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData_副本\part_3.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData_副本\part_4.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData_副本\part_5.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData_副本\part_6.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData_副本\part_7.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData_副本\part_8.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData_副本\part_9.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData_副本\part_10.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData_副本\part_11.xlsx",
]

# 遍历所有 .xlsx 文件
for file_path in file_paths:
    if file_path.endswith('.xlsx'):
        file_name = os.path.basename(file_path)
        wb = load_workbook(file_path)
        ws = wb.active

        # 获取标题行，找出“待翻译(译)”和“参考语言(英文)”列的列号
        header = {cell.value: idx for idx, cell in enumerate(ws[1], start=1)}
        translate_col = header.get("待翻译(译)")
        english_col = header.get("参考语言(英文)")

        if not translate_col or not english_col:
            print(f"[跳过] 文件 {file_name} 缺少所需的列。")
            continue

        # 遍历数据行（从第2行开始）
        for row in ws.iter_rows(min_row=2):
            translate_cell = row[translate_col - 1]
            english_cell = row[english_col - 1]
            if (translate_cell.value is None or str(translate_cell.value).strip() == '') and english_cell.value:
                translate_cell.value = english_cell.value

        # 保存文件
        wb.save(file_path)
        print(f"[完成] 文件已处理并保存：{file_name}")


[完成] 文件已处理并保存：part_1.xlsx
[完成] 文件已处理并保存：part_2.xlsx
[完成] 文件已处理并保存：part_3.xlsx
[完成] 文件已处理并保存：part_4.xlsx
[完成] 文件已处理并保存：part_5.xlsx
[完成] 文件已处理并保存：part_6.xlsx
[完成] 文件已处理并保存：part_7.xlsx
[完成] 文件已处理并保存：part_8.xlsx
[完成] 文件已处理并保存：part_9.xlsx
[完成] 文件已处理并保存：part_10.xlsx
[完成] 文件已处理并保存：part_11.xlsx


In [9]:
!ffmpeg -i "C:\\Users\\wangz\\Videos\\屏幕录制\\屏幕录制 2025-04-10 103110.mp4" -filter_complex "[0:v]setpts=0.5*PTS[v];[0:a]atempo=2.0[a]" -map "[v]" -map "[a]" output_fast.mp4

ffmpeg version 2025-03-17-git-5b9356f18e-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx